In [ ]:
# Get necessary Python packages

# Clone repos
!git clone https://github.com/paoloff/super-resolution-packages.git

# Install Python packages for Scene Text Telescope and ESPCN-PyTorch
!pip install requirements.txt

# Install Python packages for Super-image
!pip install super-image


In [1]:
# Import libraries

import numpy as np
from PIL import Image
import os
import argparse
import yaml
from IPython import embed
from easydict import EasyDict
import cv2
import sys
import time
import math
import copy
import torch
import pickle
import sys
import shutil
import random
import logging
import torchvision
import PIL
from tqdm import tqdm
import torch.nn as nn
from IPython import embed
from datetime import datetime
from time import gmtime, strftime
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter

In [2]:
# Open all images inside "data" folder and store them as np arrays
input_folder = "slide-labels-old"
output_folder = "slide-labels-new"

img_paths = [os.path.join(input_folder, img) for img in os.listdir(data_path)]

img_array = []
raw_imgs = []

for img in img_paths:
    raw = Image.open(img)
    raw_imgs.append(raw)
    img_array.append(np.array(raw))

N_images = len(img_array)

# Define output folders for each model
output_folders = {}
output_folders["stt"] = os.path.join(output_folder, "SceneTextTelescope_out")
output_folders["espcn"] = os.path.join(output_folder,"ESPCN_out")
output_folders["superimage"] = os.path.join(output_folder,"SuperImage_out")

for folder_name in output_folders.values():
    try: os.mkdir(folder_name)
    except: continue

<a id='summary'></a>

### Quick links to the models


1. [ESPCN](#espcn)

2. [Super-image](#si)

3. [Scene text telescope](#stt)

<a id='espcn'></a>
### ESPCN

Reference: https://github.com/Lornatang/ESPCN-PyTorch

[Back to quick links](#summary)

In [5]:
# Clear GPU memory
torch.cuda.empty_cache()

# Clear modules
try: 
    del sys.modules["utils"]
except:pass

try:
    del sys.modules["model"]
except: pass

# Import inference module of ESPCN
%cd ESPCN-PyTorch
from inference import *
%cd ..


c:\Users\paolo\OneDrive\Documents\Upwork\PANDA\Text resolution enhancement\Github\ESPCN-PyTorch
c:\Users\paolo\OneDrive\Documents\Upwork\PANDA\Text resolution enhancement\Github


In [6]:
# The purpose of this cell is to define arguments to main function

# The parser will do that, can leave the default parameters they are

parser = argparse.ArgumentParser()

# Model architecture name, can leave as it is
parser.add_argument("--model_arch_name",
                    type=str,
                    default="espcn_x4")

# Upscale factor, can leave as it is
parser.add_argument("--upscale_factor",
                    type=int,
                    default=4)

# List with the paths of images, can leave as it is if images are in the /data/ folder
parser.add_argument("--inputs_paths",
                    type=str,
                    default=img_paths,
                    )

# Name of output folder,  can leave as default: a folder named espcn_out inside /data/ folder
parser.add_argument("--output_folder",
                    type=str,
                    default=output_folders["espcn"],
                    )

# Model weights path, can leave as default: ESPCN_x4-T91-64bf5ee4.pth.tar inside the /models/ folder
parser.add_argument("--model_weights_path",
                    type=str,
                    default="models/ESPCN_x4-T91-64bf5ee4.pth.tar",
                    )

# Device type, default is GPU (cuda)
parser.add_argument("--device_type",
                    type=str,
                    default="cuda",
                    choices=["cpu", "cuda"])

args = parser.parse_args(args=[])


In [7]:
# Call main function
main(args)

Build `espcn_x4` model successfully.
Load `espcn_x4` model weights `c:\Users\paolo\OneDrive\Documents\Upwork\PANDA\Text resolution enhancement\Github\models\ESPCN_x4-T91-64bf5ee4.pth.tar` successfully.
SR image save to `data\ESPCN_out\+_ESPCN_out.png`
SR image save to `data\ESPCN_out\0_ESPCN_out.png`
SR image save to `data\ESPCN_out\6_ESPCN_out.png`
SR image save to `data\ESPCN_out\8_ESPCN_out.png`
SR image save to `data\ESPCN_out\alcohol_ESPCN_out.png`
SR image save to `data\ESPCN_out\BOX_ESPCN_out.png`
SR image save to `data\ESPCN_out\GARAGE_ESPCN_out.png`
SR image save to `data\ESPCN_out\GREEN_ESPCN_out.png`
SR image save to `data\ESPCN_out\I_ESPCN_out.png`
SR image save to `data\ESPCN_out\img_001_SRF_4_LR_ESPCN_out.png`
SR image save to `data\ESPCN_out\img_002_SRF_4_LR_ESPCN_out.png`
SR image save to `data\ESPCN_out\img_003_SRF_4_LR_ESPCN_out.png`
SR image save to `data\ESPCN_out\img_004_SRF_4_LR_ESPCN_out.png`
SR image save to `data\ESPCN_out\img_005_SRF_4_LR_ESPCN_out.png`
SR ima

<a id='si'></a>

### Super-image

Reference: https://github.com/eugenesiow/super-image?tab=readme-ov-file

[Back to quick links](#summary)

In [8]:
# Clear GPU memory
torch.cuda.empty_cache()

# From the catalogue of models inside the super-image package, I chose the one called dlrn-bam since it has the best overall performance
from super_image import DrlnModel, ImageLoader

model_si = DrlnModel.from_pretrained('eugenesiow/drln-bam', scale=4).to("cuda")
output_folder = output_folders["superimage"]


c:\Users\paolo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:659: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


https://huggingface.co/eugenesiow/drln-bam/resolve/main/pytorch_model_4x.pt


In [9]:
# Apply model to each image and store results in a list
outs = []

# This may take 2-3 minutes
for i, img in enumerate(raw_imgs):
    inputs = ImageLoader.load_image(img).to("cuda")
    outs.append(model_si(inputs).to("cpu"))
    if i%5 == 0:
        print(f"{i}/{N_images} images converted by Super-Image model.")


c:\Users\paolo\anaconda3\Lib\site-packages\super_image\data\loader.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  return torch.as_tensor([lr])


0/25 images converted by Super-Image model.
5/25 images converted by Super-Image model.
10/25 images converted by Super-Image model.
15/25 images converted by Super-Image model.
20/25 images converted by Super-Image model.


In [10]:
# Save all processed images to the output folder
for out, input_path in zip(outs,img_paths):
    head, tail = os.path.split(input_path)
    output_head = os.path.join(head, output_folder)
    output_path = os.path.join(output_head, tail.split(".")[0]+"_SuperImage_out.png")
    ImageLoader.save_image(out, output_path)

<a id='stt'></a>
### Scene text telescope

Reference: https://github.com/FudanVI/FudanOCR/tree/main/scene-text-telescope

[Back to quick links](#summary)

In [8]:
# Clear GPU memory
torch.cuda.empty_cache()

# Clear modules
try: 
    del sys.modules["utils"]
except:pass

try:
    del sys.modules["model"]
except: pass

# Import all the necessary STT modules
%cd scene-text-telescope
from interfaces import base
from utils import utils_moran
from interfaces.super_resolution import TextSR
from utils.meters import AverageMeter
from utils.metrics import get_str_list, Accuracy
from utils import util, ssim_psnr
from utils.util import str_filt


to_pil = transforms.ToPILImage()
to_tensor = transforms.ToTensor()

c:\Users\paolo\OneDrive\Documents\Upwork\PANDA\Text resolution enhancement\Github\scene-text-telescope


In [4]:
# The purpose of this cell is to define arguments to main function

# The parser will do that, can leave the default parameters they are
parser = argparse.ArgumentParser(description='')
parser.add_argument('--arch', default='tbsrn', choices=['tbsrn', 'tsrn', 'bicubic', 'srcnn', 'vdsr', 'srres', 'esrgan', 'rdn',
                                                        'edsr', 'lapsrn'])
parser.add_argument('--text_focus', action='store_true')
parser.add_argument('--exp_name', required=False, help='Type your experiment name')
parser.add_argument('--test', action='store_true', default=False)
parser.add_argument('--test_data_dir', type=str, default='./dataset/mydata/test')
parser.add_argument('--batch_size', type=int, default=128, help='')
parser.add_argument('--resume', type=str, default='', help='')
parser.add_argument('--rec', default='crnn', choices=['crnn'])
parser.add_argument('--STN', action='store_true', default=True, help='')
parser.add_argument('--syn', action='store_true', default=False, help='use synthetic LR')
parser.add_argument('--mixed', action='store_true', default=False, help='mix synthetic with real LR')
parser.add_argument('--mask', action='store_true', default=False, help='')
parser.add_argument('--hd_u', type=int, default=32, help='')
parser.add_argument('--srb', type=int, default=5, help='')
parser.add_argument('--demo', action='store_true', default=False)
parser.add_argument('--demo_dir', type=str, default='./demo')
args = parser.parse_args(args=[])

# There are additional configurations needed by the model which are located inside the config/super_resolution.yaml file, no need to change anything
config_path = os.path.join('config', 'super_resolution.yaml')
config = yaml.load(open(config_path, 'r'), Loader=yaml.Loader)
config = EasyDict(config)



In [5]:
# Initialize the model and importing the weights from /model/stt_pre_trained_weights.pth

Mission = TextSR(config, args)
model_dict = Mission.generator_init()
model, image_crit = model_dict['model'], model_dict['crit']
%cd ..
weights = torch.load("models/stt_pre_trained_weights.pth")
model.load_state_dict(weights['state_dict_G'])


Clean the old checkpoint None
Namespace(arch='tbsrn', text_focus=False, exp_name=None, test=False, test_data_dir='./dataset/mydata/test', batch_size=128, resume='', rec='crnn', STN=True, syn=False, mixed=False, mask=False, hd_u=32, srb=5, demo=False, demo_dir='./demo')
Total Parameters 3210335
c:\Users\paolo\OneDrive\Documents\Upwork\PANDA\Text resolution enhancement\Github


<All keys matched successfully>

In [9]:
# Re-size the images into 3x16x64 tensors (necessary step for STT)

resized_imgs = []
for img in img_array:
    resized_imgs.append(to_tensor(to_pil(cv2.resize(img, (64,16), interpolation=cv2.INTER_CUBIC))))

resized_imgs_tensor = torch.stack(resized_imgs)

In [15]:
# Apply the model to enhance each image and saving to output folder

# Switch to evaluation mode
model.eval()

# We do the computations on the GPU to speed them up. However, they can be done in cpu also
with torch.no_grad():
    outs_torch = model(resized_imgs_tensor.to("cuda")).to("cpu")

outs = []
for out in outs_torch:
    outs.append(to_pil(out))

output_folder = output_folders["stt"]

# Save all processed images to the output folder
for out, input_path in zip(outs, img_paths):
    head, tail = os.path.split(input_path)
    output_head = os.path.join(head, output_folder)
    output_path = os.path.join(output_head, tail.split(".")[0]+"_SceneTextTelescope_out.png")
    out.save(output_path)